## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import API_key

# Configure gmaps API key
gmaps.configure(API_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Oparino,RU,59.8518,48.2783,34.59,97,81,3.87,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,74.82,75,17,1.70,light rain
2,2,Fairbanks,US,64.8378,-147.7164,67.48,33,75,16.11,broken clouds
3,3,La Primavera,CO,5.4906,-70.4092,76.66,90,84,3.76,broken clouds
4,4,Lebu,CL,-37.6167,-73.6500,47.68,82,37,16.51,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature? "))
max_temp = float(input("What is the maximum temperature? "))

What is the minimum temperature? 10
What is the maximum temperature? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Oparino,RU,59.8518,48.2783,34.59,97,81,3.87,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,74.82,75,17,1.70,light rain
2,2,Fairbanks,US,64.8378,-147.7164,67.48,33,75,16.11,broken clouds
3,3,La Primavera,CO,5.4906,-70.4092,76.66,90,84,3.76,broken clouds
4,4,Lebu,CL,-37.6167,-73.6500,47.68,82,37,16.51,scattered clouds
5,5,Thompson,CA,55.7435,-97.8558,60.96,36,40,10.36,scattered clouds
6,6,Hobart,AU,-42.8794,147.3294,50.25,93,40,8.05,scattered clouds
7,7,Tuktoyaktuk,CA,69.4541,-133.0374,33.80,80,75,9.22,broken clouds
8,8,Zhangye,CN,38.9342,100.4517,68.50,31,88,2.48,overcast clouds
9,9,Summerside,CA,46.3959,-63.7876,59.13,36,77,16.11,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                449
City                   449
Country                445
Lat                    449
Lng                    449
Max Temp               449
Humidity               449
Cloudiness             449
Wind Speed             449
Current Description    449
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()
clean_travel_cities.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Oparino,RU,59.8518,48.2783,34.59,97,81,3.87,broken clouds
1,1,Rikitea,PF,-23.1203,-134.9692,74.82,75,17,1.70,light rain
2,2,Fairbanks,US,64.8378,-147.7164,67.48,33,75,16.11,broken clouds
3,3,La Primavera,CO,5.4906,-70.4092,76.66,90,84,3.76,broken clouds
4,4,Lebu,CL,-37.6167,-73.6500,47.68,82,37,16.51,scattered clouds


In [7]:
 # 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Oparino,RU,34.59,broken clouds,59.8518,48.2783,
1,Rikitea,PF,74.82,light rain,-23.1203,-134.9692,
2,Fairbanks,US,67.48,broken clouds,64.8378,-147.7164,
3,La Primavera,CO,76.66,broken clouds,5.4906,-70.4092,
4,Lebu,CL,47.68,scattered clouds,-37.6167,-73.6500,
5,Thompson,CA,60.96,scattered clouds,55.7435,-97.8558,
6,Hobart,AU,50.25,scattered clouds,-42.8794,147.3294,
7,Tuktoyaktuk,CA,33.80,broken clouds,69.4541,-133.0374,
8,Zhangye,CN,68.50,overcast clouds,38.9342,100.4517,
9,Summerside,CA,59.13,broken clouds,46.3959,-63.7876,


In [15]:
 # 6a. Set parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": API_key
}

# 6b. Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


KeyboardInterrupt: 

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [12]:
#8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
location = clean_hotel_df[["Lat", "Lng"]]

In [14]:
 # 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(location, info_box_content=hotel_info)
figure = gmaps.figure()
figure.add_layer(marker_layer)

# 11b. Display the figure
figure

Figure(layout=FigureLayout(height='420px'))